In [41]:
cd /home/tkim/Academics/DistMEC/

/home/tkim/Academics/DistMEC


In [42]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

from classes.User import *
from classes.solver import *
from classes.Server import *
from classes.distributed_utils import *

from tqdm import tqdm, tqdm_notebook
from time import sleep
import pickle
import os
import numpy as np

In [43]:
def sum_reward_collisions(Users,T):
    save_array = np.zeros([len(Users),T])
    
    for t in range(T-1):
        for u in range(len(Users)):
            save_array[u,t] = (1 - Users[u].history_collisions[t]) * np.sum(Users[u].history_reward[t])
    
    return save_array

def extract_total_train_count(Users_w_sa):
    U, T = save_array.shape
    rwd = 0
    
    np.sum(Users_w_sa > 0.5,axis=0)
    
    np.mean()

In [76]:
# System Parameters
T = 1000
num_users = 16
num_svrs = 16
num_svrs_loc = 16
num_locs = 9
mu = np.random.uniform(low=0, high = 1, size = [num_users, num_svrs])
ceiling = 40
# w = np.random.uniform(low=0, high=1, size= [num_users, num_svrs])
space_1d_dim = 10 #4 for small, 10 for big
svr_offset_uniform = 1
max_dist = 4.0

num_explore = 1
trials = 1

# result dictionaries
result_dicts = []
performance_list = []
user_dicts = []

save_name = 'results/22_06_16_ceiling_20/'
isExist = os.path.exists(save_name)
if not isExist:
  # Create a new directory because it does not exist 
    os.makedirs(save_name)


# Generate visitng locations and server locations
usr_locs = gen_eq_locs(space_1d=space_1d_dim, nums=num_locs,offset = 1.7)

with open('traces_P_40k.pkl', 'rb') as handle:
    P_collection = pickle.load(handle)
    
with open('traces_P_40k_dict.pkl', 'rb') as handle:
    P_dict = pickle.load(handle)

In [77]:
# Recording Mechanism
reward_worst_dict = {}
collisions_worst_dict = {}

reward_rsv_dict = {}
collisions_rsv_dict = {}

reward_opt_stat = {}

    
for tri in range(trials):
    svr_locs = gen_rand_locs(space_1d=space_1d_dim - svr_offset_uniform, nums=num_svrs_loc)
    # Create Users
    Users_w = []
    for i in range(num_users):
        P_temp = P_collection
        Users_w += [User(usr_locs,svr_locs,mu[i],i,
                      max_dist = max_dist, threshold_dist = 6, self_weight = 1 - 0.85, P = P_temp, ceiling = 1,
                      sticky_mode = True, kick_mode=True)]

    GoT_Users = []
    for i in range(num_users):
        GoT_Users += [GoT_User(usr_locs, svr_locs, mu[i], i, 
                     max_dist = max_dist, threshold_dist = 6, self_weight = 1 - 0.95, P = Users_w[i].P,
                     c1 = 5000, c2 = 6000, c3 = 6000, delta = 0, rho = 0.5, epsilon = 0.1,
                     c = num_users, horizon = T)]
    
    # Create Servers
    Servers = []
    for i in range(num_svrs):
        Servers += [Server(svr_locs[i],mu,i)]
    
    # Recorded Values - reservation
    regret_g = np.zeros(T) # Got
    collision_count_g = np.zeros(T)
    optimal_reward_g = np.zeros(T)

    regret_w = np.zeros(T) # Worst
    collision_count_w = np.zeros(T)
    optimal_reward = np.zeros(T)
    
    # Explore rounds are common/shared across all users
    explore_rounds(Users_w, num_users, Servers, mu, regret_w, collision_count_w, optimal_reward,
                   usr_move_flag = True, rounds=num_explore, skip_optimal = True)
    explore_rounds(GoT_Users, num_users, Servers, mu, regret_w, collision_count_g, optimal_reward,
                   usr_move_flag = True, rounds=num_explore, skip_optimal = True)

    round_start = ((num_svrs)*num_explore)+1
    copy_usr_loc(Users_w, GoT_Users)
    
    # Other reward recodings
    Users_rsv = copy.deepcopy(Users_w)
    regret_rsv = copy.deepcopy(regret_w)
    collision_count_rsv = copy.deepcopy(collision_count_w)
    for u in Users_rsv:
        u.sticky_mode = True
        u.kick_mode = True #false
        u.ceiling = ceiling
        u.expected_time_true = u.get_expected_time()

    # Centralized - true w learning
    rewards_record_ct, pulls_record_ct, ucb_ct = extract_centralized_case(Users_w, num_users, num_svrs)
    regret_ct = copy.deepcopy(regret_w)
    
    # Centralized - stationary w learning
    rewards_record_cs, pulls_record_cs, ucb_cs = extract_centralized_case(Users_w, num_users, num_svrs)
    regret_cs = copy.deepcopy(regret_w)

    # Centralized - stationary w, known mu
    w_stat = obtain_w_stationary(Users_w, num_users, num_svrs)
    optimal_stat_arms, optimal_stat_reward = offline_optimal_action(w_stat,mu)
    regret_cst = copy.deepcopy(regret_w)

    for (zzz,t) in zip(tqdm_notebook(range(T-round_start)),range(round_start, T)):
        w = obtain_w(Users_w, len(Users_w), len(Servers))
#         optimal = offline_optimal_action(w, mu)
#         optimal_reward[t] = optimal[1]
        optimal = optimal_stat_arms, optimal_stat_reward
        
        # Distributed solution 
        play_round(Users_w, Servers, mu, regret_w, collision_count_w, 
                   usr_move_flag = True, debugger = False, reservation_mode = True, optimal =optimal)
        play_round(Users_rsv, Servers, mu, regret_rsv, collision_count_rsv, 
                   usr_move_flag = False, debugger = False, reservation_mode = True, optimal =optimal, t = t)
        play_round(GoT_Users, Servers, mu, regret_g, collision_count_g, 
                   usr_move_flag = False, debugger=False, reservation_mode = False, optimal=optimal, t = t,
                  arms_override = optimal_stat_arms)
        copy_usr_loc(Users_w, GoT_Users)
        copy_usr_loc(Users_w, Users_rsv)

#         # Centralized Solution -  true w learning
#         ucb_ct = update_ucb(rewards_record_ct, pulls_record_ct, ucb_ct, t, 1)
#         arms = offline_optimal_action(w, ucb_ct)[0]
#         rewards_record_ct, pulls_record_ct = pull_super_arm(arms, mu, rewards_record_ct, pulls_record_ct)
#         regret_ct[t] = optimal[1] - expected_reward(arms, mu, w)
        
        # Centralized Solution -  stationary w learning
#         ucb_cs = update_ucb(rewards_record_cs, pulls_record_cs, ucb_cs, t, 1)
#         arms = offline_optimal_action(w_stat, ucb_cs)[0]
#         rewards_record_cs, pulls_record_cs = pull_super_arm(arms, mu, rewards_record_cs, pulls_record_cs)
#         regret_cs[t] = optimal[1] - expected_reward(arms, mu, w)
        
        # centralized solution - stationary w, known mu
#         regret_cst[t] = optimal[1] - expected_reward(optimal_stat_arms, mu, w) 

#     regret_centralized_dict[tri] = copy.deepcopy(regret_ct)

#     regret_worst_dict[tri] = copy.deepcopy(regret_w)
#     collisions_worst_dict[tri] = copy.deepcopy(collision_count_w)

#     regret_rsv_dict[tri] = copy.deepcopy(regret_rsv)
#     collisions_rsv_dict[tri] = copy.deepcopy(collision_count_rsv)

#     regret_cent_stat[tri] = copy.deepcopy(regret_cs)
#     regret_opt_stat[tri] = copy.deepcopy(regret_cst)

#     # Alter GoT data to be algorithm accurate
#     regret_g[11000:] = regret_cs[11000:]
#     collision_count_g[11000:] = 0
# #     collision_count_g[5000:11000] = 0.5
    
#     regret_got[tri] = copy.deepcopy(regret_g)
#     collisions_got[tri] = copy.deepcopy(collision_count_g)
    
#     del Users_w, Users_rsv, GoT_Users, Servers

In [78]:
Users_w_sa = sum_reward_collisions(Users_w,T)
Users_rsv_sa = sum_reward_collisions(Users_rsv,T)
GoT_Users_sa = sum_reward_collisions(GoT_Users,T)

In [79]:
print(np.sum(Users_w_sa))
print(np.sum(Users_rsv_sa))
print(np.sum(GoT_Users_sa))


1398.2559230925594
3663.984832375011
2078.9699618158074


In [80]:
np.mean(Users_w[0].reward_scale)

0.1262154939381484

In [81]:
0.7*0.5 * len(Users_w)*T

5600.0

In [99]:
np.sum(Users_w_sa[15] > 0.5)

45

In [105]:
print(np.mean(np.sum(Users_w_sa > 0.5,axis=1)))
print(np.mean(np.sum(Users_rsv_sa > 0.5,axis=1)))
print(np.mean(np.sum(GoT_Users_sa > 0.5,axis=1)))


122.125
302.3125
180.3125
